In [64]:
!which python

/Users/apabook/Desktop/ds-deployment/.venv/bin/python


In [65]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete dataspace flow

## Consts


In [66]:
data_space_provider = "http://0.0.0.0:1200"
data_space_consumer = "http://0.0.0.0:1100"
api_endpoint = "https://jsonplaceholder.typicode.com/comments"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Participants onboarding

### Current Participants information

In [67]:
url = data_space_provider + "/api/v1/participants"
payload = {
    "participantType": "Provider",
    "participantBaseUrl": "http://host.docker.internal:1200",
    "extraFields": {}
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_provider_participant_id = response_as_json["participant_id"]
data_space_provider_participant_id

'urn:uuid:5e856880-73fa-46b2-97db-6a7b89e8cda9'

In [68]:
url = data_space_provider + "/api/v1/participants"
payload = {
    "participantType": "Consumer",
    "participantBaseUrl": "http://host.docker.internal:1100",
    "extraFields": {
        "did": "did:web:asdnoiasdnoaisdnoisa:com"
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json["participant_id"]
data_space_consumer_participant_id

'urn:uuid:4b946d54-9b24-47ad-94fe-9a0fa769088b'

## Create Catalog instances and metadata

In [69]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

'urn:uuid:f6346d7b-af3a-4c87-b27a-b00b49b2a647'

In [70]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:48e14d2b-a2b9-4892-a508-851fa249023a'

In [71]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": api_endpoint
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

'urn:uuid:11c28952-b956-4bac-9c93-98c3e1e91868'

In [72]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["@id"]
distribution_id

'urn:uuid:a2487ac5-836e-4cdf-a421-21e45b8888f8'

## Create policies

In [73]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }

    ],
    "obligation": [],
    "prohibition": []
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

{
    "@id": "urn:uuid:e2b5b0fe-acd1-4b74-aac9-9049b97d8305",
    "permission": [
        {
            "action": "use",
            "constraint": [
                {
                    "rightOperand": "user",
                    "leftOperand": "did:web:hola.es",
                    "operator": "eq"
                }
            ]
        }
    ],
    "obligation": [],
    "@type": "Offer",
    "prohibition": [],
    "target": "urn:uuid:48e14d2b-a2b9-4892-a508-851fa249023a"
}


## Contract Negotiation

In [74]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerAddress": "http://host.docker.internal:1200",
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
  "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
  "offer": {
    "@id": "urn:uuid:e2b5b0fe-acd1-4b74-aac9-9049b97d8305",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:48e14d2b-a2b9-4892-a508-851fa249023a"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
    "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
    "state": "REQUESTED"
  }
}


In [75]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:4b946d54-9b24-47ad-94fe-9a0fa769088b",
  "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
  "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
  "offer": {
    "@id": "urn:uuid:e2b5b0fe-acd1-4b74-aac9-9049b97d8305",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:48e14d2b-a2b9-4892-a508-851fa249023a"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
    "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
    "state": "OFFERED"
  }
}


In [76]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerAddress": "http://host.docker.internal:1200",
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
  "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
  "offer": {
    "@id": "urn:uuid:e2b5b0fe-acd1-4b74-aac9-9049b97d8305",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:48e14d2b-a2b9-4892-a508-851fa249023a"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
    "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
    "state": "REQUESTED"
  }
}


In [77]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:4b946d54-9b24-47ad-94fe-9a0fa769088b",
  "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
  "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
  "offer": {
    "@id": "urn:uuid:e2b5b0fe-acd1-4b74-aac9-9049b97d8305",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:48e14d2b-a2b9-4892-a508-851fa249023a"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
    "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
    "state": "OFFERED"
  }
}


In [78]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerAddress": "http://host.docker.internal:1200",
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
  "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
    "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
    "state": "ACCEPTED"
  }
}


In [79]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
  "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
  "agreement": {
    "@id": "urn:uuid:d31e543b-f35b-4c5d-a378-9b39fa6bde86",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Agreement",
    "target": "urn:uuid:48e14d2b-a2b9-4892-a508-851fa249023a",
    "assigner": "urn:uuid:5e856880-73fa-46b2-97db-6a7b89e8cda9",
    "assignee": "urn:uuid:4b946d54-9b24-47ad-94fe-9a0fa769088b",
    "timestamp": "2025-06-19T07:32:15.619952250+00:00"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
    "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
    "state": "AGREED"
  }
}


In [80]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerAddress": "http://host.docker.internal:1200",
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
  "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
    "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
    "state": "VERIFIED"
  }
}


In [81]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
  "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:9a49023c-ba75-4fe1-adbe-064219ae73c0",
    "consumerPid": "urn:uuid:1151519c-ec49-492d-913f-e686c0bbae1e",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [82]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerAddress": "http://host.docker.internal:1200",
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
callback_address = callback_address.replace("0.0.0.0", "host.docker.internal")
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
  "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
  "agreementId": "urn:uuid:d31e543b-f35b-4c5d-a378-9b39fa6bde86",
  "callbackAddress": "http://0.0.0.0:1100/urn:uuid:d88f8017-5254-4eb9-8d97-377355cdd923",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
    "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
    "state": "REQUESTED"
  }
}


In [83]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
  "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "http",
    "endpoint": "http://0.0.0.0:1200/data/urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": ""
      },
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": ""
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
    "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
    "state": "STARTED"
  }
}


In [84]:
url = data_space_provider + "/api/v1/dataplane/" + transfer_provider_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = response_as_json["process_address"]["url"]
print(json.dumps(response_as_json, indent=2))

{
  "id": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
  "process_direction": "PULL",
  "upstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "downstream_hop": {
    "protocol": "https",
    "url": "https://jsonplaceholder.typicode.com/comments",
    "auth_type": "",
    "auth_content": ""
  },
  "process_address": {
    "protocol": "http",
    "url": "http://0.0.0.0:1200/data/urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
    "auth_type": "",
    "auth_content": ""
  },
  "created_at": "2025-06-19T07:32:15.799201",
  "updated_at": "2025-06-19T07:32:15.903827",
  "state": "STARTED"
}


In [85]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerAddress": "http://host.docker.internal:1200",
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
  "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
    "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
    "state": "SUSPENDED"
  }
}


In [86]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerAddress": "http://host.docker.internal:1200",
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
  "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
    "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
    "state": "STARTED"
  }
}


In [87]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
  "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
    "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
    "state": "SUSPENDED"
  }
}


In [88]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
  "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:1acc5fa2-4fb6-40be-924e-2d213024136b",
    "consumerPid": "urn:uuid:8705b9b8-92ea-434f-bd10-0096d2485cf4",
    "state": "COMPLETED"
  }
}
